<a href="https://colab.research.google.com/github/nagamamo/UofMalaya_Seminar/blob/main/4_Python_for_Finance%3A_Multi_Factor_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4 Python for Finance: Multi Factor Model**

1.   **CAPM≒Single Factor Model** estimates the expected return and the risk of individual stocks with a single market portfolio.
2.   Therefore, how to overcome the **CAPM anomaly** is one of the most important missions in a market.
3.   One of the solutions is “***Arbitrage Price Theory***” by Prof. Stephen Ross.
4.   The **CAPM anomaly** is caused by the **small size effect**, **value effect**, and **momentum effect**, etc.
1.   By controlling the above three **anomalies**, it is possible to predict the stock price of each individual stocks.

# 4.1 Installing Package
First, install the package to obtain the proxy variables for the **small size** **effect** and **value effect** required for the three-factor model.


In [ ]:
pip install getFamaFrenchFactors

# 4.2 Estimating Multi Factor Model: Libraries
In this example, we import five libraties.

In [2]:
#[1]Import Libraries
import pandas as pd
import pandas_datareader as data
import datetime
import statsmodels.api as sm
import getFamaFrenchFactors as gff

# 4.3 Estimating Multi Factor Model: Getting Data
Next, we obtain the daily stock prices.

In [3]:
#[2]Getting Data
tickers=['TSLA.US','GM.US']
start=datetime.date(2012,1,1)
end=datetime.date(2022,12,31)
df=data.DataReader(tickers,'stooq',start=start,end=end).sort_values(by='Date',ascending=True)

# 4.4 Estimating Multi Factor Model: Data Pre-processing
Store only the variable '**Close**' in dataframe df in a new dataframe.
Next, change the colum name from [‘**Close**’] to [‘**Tesla**’,‘**GM**’] .
Instruct to change the column name (variable name)  to [‘**Tesla**’,‘**GM**’] on the third line.

In [ ]:
#[3] Data Pre-processing
df = df['Close']
company_list=['Tesla','GM']
df.columns = company_list
df.describe()
df

# 4.4 Estimating Multi Factor Model: Getting Anomaly Data
Get the following 3 variables from the library **getFamaFrenchFactors** (**gff**).

**Mkt-RF**: Market Portfolio Return minus Risk Free Asset Return

**SMB**: Proxy Variable for Small Size Effect

**HML**: Proxy Variable for Value Effect





In [6]:
#[3] Getting Anomaly Data
Fama_French_3=gff.famaFrench3Factor(frequency='m')
Fama_French_3.rename(columns={"date_ff_factors":'Date'},inplace=True)
Fama_French_3.set_index('Date',inplace=True)
Returns=df.resample('M').last().pct_change().dropna()
Fama_French_data=Fama_French_3.merge(Returns,on='Date')
#Fama_French_data

# 4.5 Estimating Multi Factor Model: Tesla
Finally, we estimate the **β value**, **small size effect**, and **value effect**.
Independent variables of ***X*** are the **market portfolio return** (**Mkt-RF**), **small size effect** proxy (**SMB**), and **value effect** proxy(**HML**)
Dependent variable of ***Y*** is the market return of Tesla (**TSLA.US**).

In [7]:
#[4] Estimating Multi Factor Model: Tesla
X=Fama_French_data[['Mkt-RF','SMB','HML']]
#X=Fama_French_data[['Mkt-RF']]
y=Fama_French_data['Tesla']-Fama_French_data['RF']
X=sm.add_constant(X)
Fama_French_Model1=sm.OLS(y,X).fit()
print(Fama_French_Model1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     12.44
Date:                Sat, 15 Jun 2024   Prob (F-statistic):           3.49e-07
Time:                        05:25:48   Log-Likelihood:                 51.923
No. Observations:                 131   AIC:                            -95.85
Df Residuals:                     127   BIC:                            -84.35
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0287      0.015      1.924      0.0

# 4.6 Estimating Multi Factor Model: GM
Looking at the results, we can see that Tesla has a **growth effect**, and GM has a **small size effect** and **value effect**.

A further important finding is that due to these **CAPM anomalies**, the **β value** differs significantly between **single-factor** and **multi-factor models**.


In [8]:
#[5]Estimating Multi Factor Model: GM
X=Fama_French_data[['Mkt-RF','SMB','HML']]
z=Fama_French_data['GM']-Fama_French_data['RF']
X=sm.add_constant(X)
Fama_French_Model2=sm.OLS(z,X).fit()
print(Fama_French_Model2.summary())
#Fama_French_Model2.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     47.93
Date:                Sat, 15 Jun 2024   Prob (F-statistic):           8.76e-21
Time:                        05:25:56   Log-Likelihood:                 180.48
No. Observations:                 131   AIC:                            -353.0
Df Residuals:                     127   BIC:                            -341.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0037      0.006     -0.663      0.5